In [2]:
from __future__ import division
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.base import BaseEstimator, ClassifierMixin
import pandas as pd

#An implementation of a neural network. It is possible to use GridSearch from scikit-learn to optimize hyperparameters, 
#for example the number of hidden layers, and the learning rate. This implementation is based on Andrew NG's online lectures
#on neural networks, https://class.coursera.org/ml-003/lecture, accessed in June 2016.
class neural_network(BaseEstimator, ClassifierMixin):
    def __init__(self,number_of_hidden_layers = 2,learning_rate = 0.7,initial_weight_limit = 0.7,tuning_parameter = 0,iterations = 1200,derivative_check = False):
        #Set the number of iterations
        self.iterations = iterations
        #Set the learning rate
        self.learning_rate = learning_rate
        #Set the tuning parameter
        self.tuning_parameter = tuning_parameter
        #Set the number of hidden layers. Minimum number is 1
        self.number_of_hidden_layers = number_of_hidden_layers
        #Set the range for the initial value of the weights
        self.initial_weight_limit = initial_weight_limit
        #Set the derivative_check variable
        self.derivative_check = derivative_check
        #If the derivative check is True, calculate numerical derivatives
        if self.derivative_check == True:
        #Initialize the numerical derivative list. we will use it to check if the 
        #calculation of the derivatives is correct.
            self.numerical_derivative = []
            self.weights_output_derivative_test = []
            #Set the epsilon value that will be used to calculate the numerical derivative        
            self.epsilon = 0.00001
    def get_params(self, deep=True):
        return {"iterations": self.iterations, "learning_rate": self.learning_rate,"tuning_parameter": self.tuning_parameter,"number_of_hidden_layers": self.number_of_hidden_layers,"initial_weight_limit": self.initial_weight_limit}
    def set_params(self, **parameters):
        for parameter, value in parameters.items():
            setattr(self,parameter, value)
        return self
    def fit(self,X,y):
        self.error = []
        y = np.reshape(y,(len(y),1))
        intercept = np.ones(len(X))
        #Scale the data to have mean 0 and standard deviation 1
        self.mean = np.array(np.mean(X,axis = 0))
        self.standard_deviation = np.std(X, axis = 0)
        X = preprocessing.scale(X)
        X = np.c_[X,intercept]
        self.data = np.array(X)
        self.target = np.array(y)
        #Set the number of nodes in the hidden layers. 
        #The number of nodes in layers excludes the bias term
        self.nodes_in_hidden_layers = np.shape(X)[1]//2
        #Start the weight matrices. The dimensions of the first and last weights are different
        #then the middle weights   
        np.random.seed(1)
        self.weights_first = 2 * np.random.uniform(size = (np.shape(self.data)[1],self.nodes_in_hidden_layers)) * self.initial_weight_limit - self.initial_weight_limit #np.ones((np.shape(self.data)[1],self.nodes_in_hidden_layers))       
        self.weights_first_derivative = np.ones((np.shape(self.data)[1],self.nodes_in_hidden_layers))        
        if self.number_of_hidden_layers > 1:         
            self.weights_middle =  2 * np.random.uniform(size = (self.number_of_hidden_layers - 1,self.nodes_in_hidden_layers + 1,self.nodes_in_hidden_layers)) * self.initial_weight_limit - self.initial_weight_limit #np.ones((self.number_of_hidden_layers - 1,self.nodes_in_hidden_layers + 1,self.nodes_in_hidden_layers))       
            self.weights_middle_derivative = np.ones((self.number_of_hidden_layers - 1,self.nodes_in_hidden_layers + 1,self.nodes_in_hidden_layers))       
        #The last weight maps the last hidden layer to one probability score    
        self.weights_output =  2 * np.random.uniform(size = (self.nodes_in_hidden_layers + 1,1)) * self.initial_weight_limit - self.initial_weight_limit # np.ones((self.nodes_in_hidden_layers + 1,1))
        self.weights_output_derivative = np.ones((self.nodes_in_hidden_layers + 1,1))
        #Hidden layer(s). The first index is for the hidden layer number, the second is the length of the dataset,
        #and the third is the nodes in the hidden layer, plus 1 for the bias term        
        self.hidden_layers = np.zeros((self.number_of_hidden_layers,len(self.data),self.nodes_in_hidden_layers + 1))        
        #Set the bias terms to 1            
        self.hidden_layers[:,:,self.nodes_in_hidden_layers] = 1
        #Initiate the probabilities. These are the outputs of the neural network        
        self.probabilities = np.zeros((len(self.data),1))
        #Create the matrices that will be used for backpropagation
        self.delta_output = np.zeros((len(self.data),1))
        self.delta_middle = np.zeros((self.number_of_hidden_layers,len(self.data),self.nodes_in_hidden_layers))
        for i in range(self.iterations):
            #Start with forward propagation to calculate probabilities            
            self.hidden_layers[0,:,:self.nodes_in_hidden_layers] = 1/(1 + np.exp(-np.dot(self.data,self.weights_first))) 
            if self.number_of_hidden_layers > 1:
                for h in range(self.number_of_hidden_layers - 1):
                    self.hidden_layers[h + 1,:,:self.nodes_in_hidden_layers] = 1/(1 + np.exp(-np.dot(self.hidden_layers[h,:,:],self.weights_middle[h])))
            self.probabilities = 1/(1 + np.exp(-np.dot(self.hidden_layers[self.number_of_hidden_layers - 1],self.weights_output)))
            #Calculate the error (J(theta))           
            self.error.append(np.average(np.dot(self.hidden_layers[self.number_of_hidden_layers - 1],self.weights_output) * (1 - self.target) + np.log(1 + np.exp(-np.dot(self.hidden_layers[self.number_of_hidden_layers - 1],self.weights_output)))))             
            #Back propagation            
            #Calculate delta for the output layer:
            self.delta_output = self.probabilities - self.target 
            derivative = self.hidden_layers[self.number_of_hidden_layers - 1,:,:self.nodes_in_hidden_layers] * (1 - self.hidden_layers[self.number_of_hidden_layers - 1,:,:self.nodes_in_hidden_layers])
            self.delta_middle[self.number_of_hidden_layers - 1] = np.dot(self.delta_output,np.transpose(self.weights_output[:self.nodes_in_hidden_layers])) * derivative
            if self.number_of_hidden_layers > 1:
                for h in range(self.number_of_hidden_layers - 1):
                    derivative = self.hidden_layers[self.number_of_hidden_layers - h - 2,:,:self.nodes_in_hidden_layers] * (1 - self.hidden_layers[self.number_of_hidden_layers - h - 2,:,:self.nodes_in_hidden_layers])
                    self.delta_middle[self.number_of_hidden_layers - h - 2] = np.dot(self.delta_middle[self.number_of_hidden_layers - h - 1],np.transpose(self.weights_middle[self.number_of_hidden_layers - h - 2,:self.nodes_in_hidden_layers])) * derivative
            #Calculate derivative of the cost function with respect to each weight  
            self.weights_output_derivative = np.dot(np.transpose(self.hidden_layers[self.number_of_hidden_layers - 1]),self.delta_output)/len(self.data)          
            #Regularization for weights except for the weight for the bias term
            self.weights_output_derivative[:self.nodes_in_hidden_layers] = self.weights_output_derivative[:self.nodes_in_hidden_layers] + self.tuning_parameter * self.weights_output_derivative[:self.nodes_in_hidden_layers]           
            if self.derivative_check == True:
            ##Check if the derivatives are calculated correctly##            
            #We can check if the derivatives are calculated correctly, by comparing
            #them with the derivatives calculated by numerical estimation.
            #Use 2-sided difference to calculate the derivatives of the first weights
                weight_index = 1
                self.weights_output_upper = np.copy(self.weights_output)
                self.weights_output_upper[weight_index] = self.weights_output_upper[weight_index] + self.epsilon
                error_upper = np.average(np.dot(self.hidden_layers[self.number_of_hidden_layers - 1],self.weights_output_upper) * (1 - self.target) + np.log(1 + np.exp(-np.dot(self.hidden_layers[self.number_of_hidden_layers - 1],self.weights_output_upper))))
                self.weights_output_lower = np.copy(self.weights_output)
                self.weights_output_lower[weight_index] = self.weights_output_lower[weight_index] - self.epsilon            
                error_lower = np.average(np.dot(self.hidden_layers[self.number_of_hidden_layers - 1],self.weights_output_lower) * (1 - self.target) + np.log(1 + np.exp(-np.dot(self.hidden_layers[self.number_of_hidden_layers - 1],self.weights_output_lower))))            
                self.numerical_derivative.append((error_upper - error_lower)/ (2 * self.epsilon))            
                self.weights_output_derivative_test.append(self.weights_output_derivative[weight_index][0])            
            if self.number_of_hidden_layers > 1:
                for h in range(self.number_of_hidden_layers - 1):
                    self.weights_middle_derivative[self.number_of_hidden_layers - h - 2] = np.dot(np.transpose(self.hidden_layers[self.number_of_hidden_layers - h - 2]),self.delta_middle[self.number_of_hidden_layers - h - 1])/len(self.data)
                    #Regularization for weights except for the weight for the bias term
                    self.weights_middle_derivative[self.number_of_hidden_layers - h - 2,:self.nodes_in_hidden_layers,:] = self.weights_middle_derivative[self.number_of_hidden_layers - h - 2,:self.nodes_in_hidden_layers,:] + self.tuning_parameter * self.weights_middle_derivative[self.number_of_hidden_layers - h - 2,:self.nodes_in_hidden_layers,:]      
            self.weights_first_derivative = np.dot(np.transpose(self.data),self.delta_middle[0])/len(self.data)
            #Regularization for weights except for the weight for the bias term
            self.weights_first_derivative[:self.nodes_in_hidden_layers] = self.weights_first_derivative[:self.nodes_in_hidden_layers] + self.tuning_parameter * self.weights_first_derivative[:self.nodes_in_hidden_layers]
            #Update the parameters using the derivatives calculated by backpropagation
            self.weights_first = self.weights_first - self.learning_rate * self.weights_first_derivative
            if self.number_of_hidden_layers > 1:
                self.weights_middle = self.weights_middle - self.learning_rate * self.weights_middle_derivative
            self.weights_output = self.weights_output - self.learning_rate * self.weights_output_derivative            
        if self.derivative_check == True:
            print(pd.DataFrame({'numerical':self.numerical_derivative,'analytic':self.weights_output_derivative_test}))
    def predict_proba(self,newdata):
        newdata = preprocessing.scale(newdata)
        intercept = np.ones(len(newdata))
        newdata = np.c_[newdata,intercept]        
        #Hidden layer(s)        
        self.hidden_layers = np.zeros((self.number_of_hidden_layers,len(newdata),self.nodes_in_hidden_layers + 1))        
        #Set the bias terms to 1            
        self.hidden_layers[:,:,self.nodes_in_hidden_layers] = 1
        #Probabilities        
        self.probabilities_test = np.zeros((len(newdata),1))
        self.hidden_layers[0,:,:self.nodes_in_hidden_layers] = 1/(1 + np.exp(-np.dot(newdata,self.weights_first))) 
        if self.number_of_hidden_layers > 1:
            for h in range(self.number_of_hidden_layers - 1):
                self.hidden_layers[h + 1,:,:self.nodes_in_hidden_layers] = 1/(1 + np.exp(-np.dot(self.hidden_layers[h,:,:],self.weights_middle[h])))
        self.probabilities_test = 1/(1 + np.exp(-np.dot(self.hidden_layers[self.number_of_hidden_layers - 1],self.weights_output)))
        self.probabilities_test = np.ravel(self.probabilities_test)
        return self.probabilities_test        
    def predict(self,newdata):
        newdata = preprocessing.scale(newdata)
        intercept = np.ones(len(newdata))
        newdata = np.c_[newdata,intercept]        
        #Hidden layer(s)        
        self.hidden_layers = np.zeros((self.number_of_hidden_layers,len(newdata),self.nodes_in_hidden_layers + 1))        
        #Set the bias terms to 1            
        self.hidden_layers[:,:,self.nodes_in_hidden_layers] = 1
        #Start probabilities array        
        self.probabilities_test = np.zeros((len(newdata),1))
        #Start predictions array
        self.predictions_test = np.zeros((len(newdata),1))
        self.hidden_layers[0,:,:self.nodes_in_hidden_layers] = 1/(1 + np.exp(-np.dot(newdata,self.weights_first))) 
        if self.number_of_hidden_layers > 1:
            for h in range(self.number_of_hidden_layers - 1):
                self.hidden_layers[h + 1,:,:self.nodes_in_hidden_layers] = 1/(1 + np.exp(-np.dot(self.hidden_layers[h,:,:],self.weights_middle[h])))
        self.probabilities_test = 1/(1 + np.exp(-np.dot(self.hidden_layers[self.number_of_hidden_layers - 1],self.weights_output)))
        self.predictions_test = [0 if x < 0.5 else 1 for x in self.probabilities_test]
        return self.predictions_test
    def get_error_list(self):
    #Get the errors from each iteration
        return self.error